In [1]:
from astroquery.gaia import Gaia

import numpy as np
import pandas as pd
import sys
sys.path.append('../../scripts/')
from query import *

In [2]:
# Query parameters
TARGET_G_MAG_LIMIT = 6  # gs.phot_g_mean_mag < TARGET_G_MAG_LIMIT
NEIGHBOR_G_MAG_LIMIT = TARGET_G_MAG_LIMIT + 1  # neighbors.phot_g_mean_mag < NEIGHBOR_G_MAG_LIMIT
MIN_DEC = -85  # gs.dec >= MIN_DEC
MAX_DEC = 30  # gs.dec <= MAX_DEC
MIN_PARALLAX = 20  # gs.parallax >= MIN_PARALLAX (MAX_DISTANCE = 1000/MIN_PARALLAX)

In [3]:
query_all_stars = f"""
SELECT gs.source_id AS dr2_source_id,
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag, 
    gs.phot_bp_mean_mag, 
    gs.phot_rp_mean_mag, 
    gs.bp_rp, 
    gs.parallax
FROM gaiadr2.gaia_source AS gs
WHERE gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX} 
"""

no_bright_neighbor_query = f"""
SELECT 
    gs.source_id, 
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag
FROM 
    gaiadr2.gaia_source AS gs
WHERE 
    gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX}
    AND NOT EXISTS (
        SELECT 1
        FROM gaiadr2.gaia_source AS neighbors
        WHERE 
            1=CONTAINS(
                POINT('ICRS', gs.ra, gs.dec),
                CIRCLE('ICRS', neighbors.ra, neighbors.dec, 2/3600.0)
            )
            AND neighbors.phot_g_mean_mag < {NEIGHBOR_G_MAG_LIMIT}
            AND gs.source_id != neighbors.source_id
    )
"""

bright_neighbor_query = f"""
SELECT 
    gs.source_id, 
    gs.ra, 
    gs.dec, 
    gs.phot_g_mean_mag, 
    neighbors.source_id AS neighbor_id,
    neighbors.phot_g_mean_mag AS neighbor_mag,
    DISTANCE(
        POINT('ICRS', gs.ra, gs.dec), 
        POINT('ICRS', neighbors.ra, neighbors.dec)
    ) AS angular_distance
FROM 
    gaiadr2.gaia_source AS gs
JOIN 
    gaiadr2.gaia_source AS neighbors
ON 
    1=CONTAINS(
        POINT('ICRS', gs.ra, gs.dec),
        CIRCLE('ICRS', neighbors.ra, neighbors.dec, 2/3600.0)
    )
WHERE 
    gs.phot_g_mean_mag < {TARGET_G_MAG_LIMIT}
    AND gs.dec BETWEEN {MIN_DEC} AND {MAX_DEC}
    AND gs.parallax >= {MIN_PARALLAX}
    AND neighbors.phot_g_mean_mag < {NEIGHBOR_G_MAG_LIMIT}
    AND gs.source_id != neighbors.source_id
"""

# ... rest of the code remains the same ...

In [5]:


if 1: # For the all stars query
    df_all_stars = execute_gaia_query(
        query_all_stars,
        str_columns=['dr2_source_id'],
        output_file='all_stars_dr2.xlsx'
    )

if 1: # For the no bright neighbor query
    df_no_bright_neighbor = execute_gaia_query(
        no_bright_neighbor_query,
        str_columns=['source_id'],
        output_file='no_bright_neighbor_results.xlsx'
    )

if 1: # For the bright neighbor query
    df_bright_neighbor = execute_gaia_query(
        bright_neighbor_query,
        str_columns=['source_id', 'neighbor_id'],
        output_file='bright_neighbor_results.xlsx'
    )

INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 734
INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 717
INFO: Query finished. [astroquery.utils.tap.core]
Number of results: 17


### now switch to filtering the ones in merged_df

In [40]:
merged_df = pd.read_excel('../../results/combined_query_with_mass_detection_limit.xlsx', dtype={'source_id': str, 'source_id_dr2': str, 'source_id_dr3': str, 'HIP Number': str})#.iloc[8:20]
merged_df

,source_id,source_id_dr2,source_id_dr3,RA,DEC,V_mag,Phot G Mean Mag,Phot BP Mean Mag,Phot RP Mean Mag,BP-RP,...,Radius [R_Sun],HZ_limit [AU],RV precision [m/s],HZ Detection Limit [M_Earth],Spectral Type,HD Number,GJ Number,HIP Number,Object Type,HZ Detection Limit Simplified [M_Earth]
0,22269508511466624,22269508511466624,22269508511466624,41.482248,10.666519,12.687016,11.891757,12.940540,10.878490,2.062050,...,0.659257,0.259182,0.912258,3.888545,M,NaN,NaN,NaN,HighPM*,3.888505
1,1173206811240350592,1173206811240350592,1173206811240350592,221.961398,7.416476,11.617188,10.907482,11.867918,9.918146,1.949772,...,0.872650,0.343274,0.557978,2.912921,M,NaN,NaN,NaN,HighPM*,2.912901
2,2683023811628007296,2683023811628007296,2683023811628007296,330.540802,1.398992,9.136045,8.403359,9.387209,7.420015,1.967194,...,0.702667,0.276365,0.178149,0.795958,M,HD 209290,GJ 846,108782,HighPM*,0.795956
3,3919427007885527680,3919427007885527680,3919427007885527680,179.383579,11.828810,11.797612,10.899825,12.055270,9.827121,2.228149,...,0.548498,0.217990,0.608185,2.008597,M,NaN,GJ 3695,58321,HighPM*,2.008582
4,4595398441587270272,4595398441587270272,4595398441587270272,264.878042,27.761330,11.072632,10.337281,11.323845,9.354651,1.969193,...,0.757623,0.298467,0.435757,1.882718,M,NaN,GJ 4018,86423,HighPM*,1.882708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4488,4762394397915837056,4762394397915837056,4762394397915837056,73.721844,-58.547353,6.005004,6.005004,6.222638,5.630950,0.591688,...,1.241460,1.536463,1.795406,28.367441,F,HD 31746,NaN,22844,HighPM*,28.366511
4489,6323603052822806784,6323603052822806784,NaN,219.245202,-12.303777,6.062907,6.062907,6.331566,5.721863,0.609704,...,1.127712,1.316431,1.844249,26.984210,B,HD 128429,NaN,71469,HighPM*,26.983369
4490,6536790454707565696,6536790454707565696,NaN,350.491302,-40.174354,9.840427,9.840427,10.085142,9.426617,0.658525,...,1.350000,1.512488,10.696199,167.778821,NaN,NaN,NaN,NaN,NaN,167.746334
4491,1273169013960272640,1273169013960272640,1273169013960272640,234.223073,29.990933,6.356777,6.356777,6.567007,5.985064,0.581943,...,1.303444,1.529632,2.247971,34.867299,F,HD 139389,NaN,76456,HighPM*,34.865848


In [41]:
import pandas as pd
from astroquery.gaia import Gaia
import time

# Parameters
SEARCH_RADIUS = 2 / 3600.0  # 2 arcseconds in degrees

# Function to create a query for nearby stars
def create_neighbor_query(source_id, ra, dec, neighbor_g_mag_limit, search_radius, data_release):
    query = f"""
    SELECT 
        source_id, ra, dec, phot_g_mean_mag
    FROM 
        {data_release}.gaia_source
    WHERE 
        1=CONTAINS(
            POINT('ICRS', {ra}, {dec}),
            CIRCLE('ICRS', ra, dec, {search_radius})
        )
        AND phot_g_mean_mag < {neighbor_g_mag_limit}
        AND source_id != {source_id}
    """
    return query

# DataFrames: merged_df (input sources)
rows_with_bright_neighbors = []
rows_without_bright_neighbors = []

# Iterate through each row in merged_df
for i, row in merged_df.iterrows():
    print(i, row['source_id_dr2'], row['source_id_dr3'])
    
    if not pd.isna(row['source_id_dr3']):
        query = create_neighbor_query(  
            source_id=row['source_id_dr3'],
            ra=row['RA'],
            dec=row['DEC'],
            neighbor_g_mag_limit=row['Phot G Mean Mag']+3,
            search_radius=SEARCH_RADIUS,
            data_release='gaiadr3'
        )
    else:
        query = create_neighbor_query(
            source_id=row['source_id_dr2'],
            ra=row['RA'],
            dec=row['DEC'],
            neighbor_g_mag_limit=row['Phot G Mean Mag']+3,
            search_radius=SEARCH_RADIUS,
            data_release='gaiadr2'
        )
    
    # Execute the query
    neighbors_df = execute_gaia_query(query)
    
    # Check if bright neighbors exist
    if neighbors_df is not None and not neighbors_df.empty:
        # Add the row to the bright neighbors list
        rows_with_bright_neighbors.append(row)
    else:
        # Add the row to the no bright neighbors list
        rows_without_bright_neighbors.append(row)

# Create DataFrames for rows with and without bright neighbors
bright_neighbors_df = pd.DataFrame(rows_with_bright_neighbors)
rows_without_bright_neighbors_df = pd.DataFrame(rows_without_bright_neighbors)

# Output the results
print(f"Rows with bright neighbors: {len(bright_neighbors_df)}")
print(f"Rows without bright neighbors: {len(rows_without_bright_neighbors_df)}")


0 22269508511466624 22269508511466624
INFO: Query finished. [astroquery.utils.tap.core]
1 1173206811240350592 1173206811240350592
INFO: Query finished. [astroquery.utils.tap.core]
2 2683023811628007296 2683023811628007296
INFO: Query finished. [astroquery.utils.tap.core]
3 3919427007885527680 3919427007885527680
INFO: Query finished. [astroquery.utils.tap.core]
4 4595398441587270272 4595398441587270272
INFO: Query finished. [astroquery.utils.tap.core]
5 4188969431580611456 4188969431580611456
INFO: Query finished. [astroquery.utils.tap.core]
6 5702187702467261696 5702187702467261696
INFO: Query finished. [astroquery.utils.tap.core]
7 3903318372263850368 3903318372263850368
INFO: Query finished. [astroquery.utils.tap.core]
8 4975284381905517824 nan
INFO: Query finished. [astroquery.utils.tap.core]
9 5970137861534914304 5970137861534914304
INFO: Query finished. [astroquery.utils.tap.core]
10 5946986063684477056 5946986063684477056
INFO: Query finished. [astroquery.utils.tap.core]
11 3828

In [39]:
output_path = 'stars_with_bright_neighbors.xlsx'
bright_neighbors_df.to_excel(output_path, index=False)
adjust_column_widths(output_path)

output_path = 'stars_without_bright_neighbors.xlsx'
rows_without_bright_neighbors_df.to_excel(output_path, index=False)
adjust_column_widths(output_path)

